## Imports & Configuration

In [2]:
import warnings
warnings.filterwarnings("ignore")

StatementMeta(, 00754ea3-aa16-4d52-8e44-2442e3a640ab, 4, Finished, Available, Finished)

In [3]:
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

StatementMeta(, 00754ea3-aa16-4d52-8e44-2442e3a640ab, 5, Finished, Available, Finished)

In [4]:
spark = SparkSession.builder.appName("QueryPlan").getOrCreate()

StatementMeta(, 00754ea3-aa16-4d52-8e44-2442e3a640ab, 6, Finished, Available, Finished)

## Reading File

In [5]:
transaction_file = "Files/data/data_skew/transactions.parquet"
df_transactions = spark.read.parquet(transaction_file)

StatementMeta(, 00754ea3-aa16-4d52-8e44-2442e3a640ab, 7, Finished, Available, Finished)

In [6]:
df_transactions.show(5, False)

StatementMeta(, 00754ea3-aa16-4d52-8e44-2442e3a640ab, 8, Finished, Available, Finished)

+----------+----------+----------+---------------+----------+----+-----+---+-------------+------+-----------+
|cust_id   |start_date|end_date  |txn_id         |date      |year|month|day|expense_type |amt   |city       |
+----------+----------+----------+---------------+----------+----+-----+---+-------------+------+-----------+
|C0YDPQWPBJ|2010-07-01|2018-12-01|TZ5SMKZY9S03OQJ|2018-10-07|2018|10   |7  |Entertainment|10.42 |boston     |
|C0YDPQWPBJ|2010-07-01|2018-12-01|TYIAPPNU066CJ5R|2016-03-27|2016|3    |27 |Motor/Travel |44.34 |portland   |
|C0YDPQWPBJ|2010-07-01|2018-12-01|TETSXIK4BLXHJ6W|2011-04-11|2011|4    |11 |Entertainment|3.18  |chicago    |
|C0YDPQWPBJ|2010-07-01|2018-12-01|TQKL1QFJY3EM8LO|2018-02-22|2018|2    |22 |Groceries    |268.97|los_angeles|
|C0YDPQWPBJ|2010-07-01|2018-12-01|TYL6DFP09PPXMVB|2010-10-16|2010|10   |16 |Entertainment|2.66  |chicago    |
+----------+----------+----------+---------------+----------+----+-----+---+-------------+------+-----------+
only showi

In [7]:
customers_file = "Files/data/data_skew/customers.parquet"
df_customers = spark.read.parquet(customers_file)

StatementMeta(, 00754ea3-aa16-4d52-8e44-2442e3a640ab, 9, Finished, Available, Finished)

In [8]:
df_customers.show(5, False)

StatementMeta(, 00754ea3-aa16-4d52-8e44-2442e3a640ab, 10, Finished, Available, Finished)

+----------+-------------+---+------+----------+-----+-----------+
|cust_id   |name         |age|gender|birthday  |zip  |city       |
+----------+-------------+---+------+----------+-----+-----------+
|C007YEYTX9|Aaron Abbott |34 |Female|7/13/1991 |97823|boston     |
|C00B971T1J|Aaron Austin |37 |Female|12/16/2004|30332|chicago    |
|C00WRSJF1Q|Aaron Barnes |29 |Female|3/11/1977 |23451|denver     |
|C01AZWQMF3|Aaron Barrett|31 |Male  |7/9/1998  |46613|los_angeles|
|C01BKUFRHA|Aaron Becker |54 |Male  |11/24/1979|40284|san_diego  |
+----------+-------------+---+------+----------+-----+-----------+
only showing top 5 rows



### Narrow Transformation

In [9]:
df_narrow_transformation = (
    df_customers
    .filter(F.col("city") == "boston")
    .withColumn("first_name", F.split("name", " ").getItem(0))
    .withColumn("last_name",F.split("name", " ").getItem(1))
    .withColumn("age", F.col("age") + F.lit(5))
    .select("cust_id", "first_name", "last_name", "age", "gender", "birthday")
)


df_narrow_transformation.show(5, False)

StatementMeta(, 00754ea3-aa16-4d52-8e44-2442e3a640ab, 11, Finished, Available, Finished)

+----------+----------+---------+----+------+---------+
|cust_id   |first_name|last_name|age |gender|birthday |
+----------+----------+---------+----+------+---------+
|C007YEYTX9|Aaron     |Abbott   |39.0|Female|7/13/1991|
|C08XAQUY73|Aaron     |Lambert  |59.0|Female|11/5/1966|
|C094P1VXF9|Aaron     |Lindsey  |29.0|Male  |9/21/1990|
|C097SHE1EF|Aaron     |Lopez    |27.0|Female|4/18/2001|
|C0DTC6436T|Aaron     |Schwartz |57.0|Female|7/9/1962 |
+----------+----------+---------+----+------+---------+
only showing top 5 rows



In [10]:
df_narrow_transformation.explain(True)

StatementMeta(, 00754ea3-aa16-4d52-8e44-2442e3a640ab, 12, Finished, Available, Finished)

== Parsed Logical Plan ==
'Project ['cust_id, 'first_name, 'last_name, 'age, 'gender, 'birthday]
+- Project [cust_id#658, name#659, (cast(age#660 as double) + cast(5 as double)) AS age#721, gender#661, birthday#662, zip#663, city#664, first_name#702, last_name#711]
   +- Project [cust_id#658, name#659, age#660, gender#661, birthday#662, zip#663, city#664, first_name#702, split(name#659,  , -1)[1] AS last_name#711]
      +- Project [cust_id#658, name#659, age#660, gender#661, birthday#662, zip#663, city#664, split(name#659,  , -1)[0] AS first_name#702]
         +- Filter (city#664 = boston)
            +- Relation [cust_id#658,name#659,age#660,gender#661,birthday#662,zip#663,city#664] parquet

== Analyzed Logical Plan ==
cust_id: string, first_name: string, last_name: string, age: double, gender: string, birthday: string
Project [cust_id#658, first_name#702, last_name#711, age#721, gender#661, birthday#662]
+- Project [cust_id#658, name#659, (cast(age#660 as double) + cast(5 as double))

### Wide Transformations

##### 1. Repartition

In [11]:
df_transactions.rdd.getNumPartitions()

StatementMeta(, 00754ea3-aa16-4d52-8e44-2442e3a640ab, 13, Finished, Available, Finished)

12

In [12]:
df_transactions.repartition(24).explain(True)

StatementMeta(, 00754ea3-aa16-4d52-8e44-2442e3a640ab, 14, Finished, Available, Finished)

== Parsed Logical Plan ==
Repartition 24, true
+- Relation [cust_id#590,start_date#591,end_date#592,txn_id#593,date#594,year#595,month#596,day#597,expense_type#598,amt#599,city#600] parquet

== Analyzed Logical Plan ==
cust_id: string, start_date: string, end_date: string, txn_id: string, date: string, year: string, month: string, day: string, expense_type: string, amt: string, city: string
Repartition 24, true
+- Relation [cust_id#590,start_date#591,end_date#592,txn_id#593,date#594,year#595,month#596,day#597,expense_type#598,amt#599,city#600] parquet

== Optimized Logical Plan ==
Repartition 24, true
+- Relation [cust_id#590,start_date#591,end_date#592,txn_id#593,date#594,year#595,month#596,day#597,expense_type#598,amt#599,city#600] parquet

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Exchange RoundRobinPartitioning(24), REPARTITION_BY_NUM, [plan_id=325]
   +- FileScan parquet [cust_id#590,start_date#591,end_date#592,txn_id#593,date#594,year#595,month#596,day#597,expens

#### 2. Coalesce

In [13]:
df_transactions.rdd.getNumPartitions()

StatementMeta(, 00754ea3-aa16-4d52-8e44-2442e3a640ab, 15, Finished, Available, Finished)

12

In [14]:
df_transactions.coalesce(1).explain(True)

StatementMeta(, 00754ea3-aa16-4d52-8e44-2442e3a640ab, 16, Finished, Available, Finished)

== Parsed Logical Plan ==
Repartition 1, false
+- Relation [cust_id#590,start_date#591,end_date#592,txn_id#593,date#594,year#595,month#596,day#597,expense_type#598,amt#599,city#600] parquet

== Analyzed Logical Plan ==
cust_id: string, start_date: string, end_date: string, txn_id: string, date: string, year: string, month: string, day: string, expense_type: string, amt: string, city: string
Repartition 1, false
+- Relation [cust_id#590,start_date#591,end_date#592,txn_id#593,date#594,year#595,month#596,day#597,expense_type#598,amt#599,city#600] parquet

== Optimized Logical Plan ==
Repartition 1, false
+- Relation [cust_id#590,start_date#591,end_date#592,txn_id#593,date#594,year#595,month#596,day#597,expense_type#598,amt#599,city#600] parquet

== Physical Plan ==
Coalesce 1
+- *(1) ColumnarToRow
   +- FileScan parquet [cust_id#590,start_date#591,end_date#592,txn_id#593,date#594,year#595,month#596,day#597,expense_type#598,amt#599,city#600] Batched: true, DataFilters: [], Format: Parquet,

#### 3. Joins

In [15]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

StatementMeta(, 00754ea3-aa16-4d52-8e44-2442e3a640ab, 17, Finished, Available, Finished)

In [16]:
df_joined = (
    df_transactions.join(
        df_customers,
        how="inner",
        on="cust_id"
    )
)

StatementMeta(, 00754ea3-aa16-4d52-8e44-2442e3a640ab, 18, Finished, Available, Finished)

In [17]:
df_joined.explain(True)

StatementMeta(, 00754ea3-aa16-4d52-8e44-2442e3a640ab, 19, Finished, Available, Finished)

== Parsed Logical Plan ==
'Join UsingJoin(Inner, [cust_id])
:- Relation [cust_id#590,start_date#591,end_date#592,txn_id#593,date#594,year#595,month#596,day#597,expense_type#598,amt#599,city#600] parquet
+- Relation [cust_id#658,name#659,age#660,gender#661,birthday#662,zip#663,city#664] parquet

== Analyzed Logical Plan ==
cust_id: string, start_date: string, end_date: string, txn_id: string, date: string, year: string, month: string, day: string, expense_type: string, amt: string, city: string, name: string, age: string, gender: string, birthday: string, zip: string, city: string
Project [cust_id#590, start_date#591, end_date#592, txn_id#593, date#594, year#595, month#596, day#597, expense_type#598, amt#599, city#600, name#659, age#660, gender#661, birthday#662, zip#663, city#664]
+- Join Inner, (cust_id#590 = cust_id#658)
   :- Relation [cust_id#590,start_date#591,end_date#592,txn_id#593,date#594,year#595,month#596,day#597,expense_type#598,amt#599,city#600] parquet
   +- Relation [cus

#### 04. GroupBy

In [18]:
df_transactions.printSchema()

StatementMeta(, 00754ea3-aa16-4d52-8e44-2442e3a640ab, 20, Finished, Available, Finished)

root
 |-- cust_id: string (nullable = true)
 |-- start_date: string (nullable = true)
 |-- end_date: string (nullable = true)
 |-- txn_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- expense_type: string (nullable = true)
 |-- amt: string (nullable = true)
 |-- city: string (nullable = true)



In [19]:
df_city_counts = (
    df_transactions
    .groupBy("city")
    .count()
)

StatementMeta(, 00754ea3-aa16-4d52-8e44-2442e3a640ab, 21, Finished, Available, Finished)

In [20]:
df_city_counts.explain(True)

StatementMeta(, 00754ea3-aa16-4d52-8e44-2442e3a640ab, 22, Finished, Available, Finished)

== Parsed Logical Plan ==
'Aggregate ['city], ['city, count(1) AS count#791L]
+- Relation [cust_id#590,start_date#591,end_date#592,txn_id#593,date#594,year#595,month#596,day#597,expense_type#598,amt#599,city#600] parquet

== Analyzed Logical Plan ==
city: string, count: bigint
Aggregate [city#600], [city#600, count(1) AS count#791L]
+- Relation [cust_id#590,start_date#591,end_date#592,txn_id#593,date#594,year#595,month#596,day#597,expense_type#598,amt#599,city#600] parquet

== Optimized Logical Plan ==
Aggregate [city#600], [city#600, count(1) AS count#791L]
+- Project [city#600]
   +- Relation [cust_id#590,start_date#591,end_date#592,txn_id#593,date#594,year#595,month#596,day#597,expense_type#598,amt#599,city#600] parquet

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[city#600], functions=[count(1)], output=[city#600, count#791L])
   +- Exchange hashpartitioning(city#600, 200), ENSURE_REQUIREMENTS, [plan_id=393]
      +- HashAggregate(keys=[city#600], 

In [21]:
df_txn_amt_city = (
    df_transactions
    .groupBy("city")
    .agg(F.sum("amt").alias("txn_amt"))
)

StatementMeta(, 00754ea3-aa16-4d52-8e44-2442e3a640ab, 23, Finished, Available, Finished)

In [22]:
df_txn_amt_city.explain(True)

StatementMeta(, 00754ea3-aa16-4d52-8e44-2442e3a640ab, 24, Finished, Available, Finished)

== Parsed Logical Plan ==
'Aggregate ['city], ['city, sum('amt) AS txn_amt#809]
+- Relation [cust_id#590,start_date#591,end_date#592,txn_id#593,date#594,year#595,month#596,day#597,expense_type#598,amt#599,city#600] parquet

== Analyzed Logical Plan ==
city: string, txn_amt: double
Aggregate [city#600], [city#600, sum(cast(amt#599 as double)) AS txn_amt#809]
+- Relation [cust_id#590,start_date#591,end_date#592,txn_id#593,date#594,year#595,month#596,day#597,expense_type#598,amt#599,city#600] parquet

== Optimized Logical Plan ==
Aggregate [city#600], [city#600, sum(cast(amt#599 as double)) AS txn_amt#809]
+- Project [amt#599, city#600]
   +- Relation [cust_id#590,start_date#591,end_date#592,txn_id#593,date#594,year#595,month#596,day#597,expense_type#598,amt#599,city#600] parquet

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[city#600], functions=[sum(cast(amt#599 as double))], output=[city#600, txn_amt#809])
   +- Exchange hashpartitioning(city#600, 200),

#### 05. GroupBy Count Distinct

In [23]:
df_txn_per_city = (
    df_transactions
    .groupBy("cust_id")
    .agg(F.countDistinct("city").alias("city_count"))
)

StatementMeta(, 00754ea3-aa16-4d52-8e44-2442e3a640ab, 25, Finished, Available, Finished)

In [24]:
df_txn_per_city.show(5, False)
df_txn_per_city.explain(True)

StatementMeta(, 00754ea3-aa16-4d52-8e44-2442e3a640ab, 26, Finished, Available, Finished)

+----------+----------+
|cust_id   |city_count|
+----------+----------+
|CPP8BY8U93|10        |
|CYB8BX9LU1|10        |
|CFRT841CCD|10        |
|CA0TSNMYDK|10        |
|COZ8NONEVZ|10        |
+----------+----------+
only showing top 5 rows

== Parsed Logical Plan ==
'Aggregate ['cust_id], ['cust_id, 'count(distinct 'city) AS city_count#826]
+- Relation [cust_id#590,start_date#591,end_date#592,txn_id#593,date#594,year#595,month#596,day#597,expense_type#598,amt#599,city#600] parquet

== Analyzed Logical Plan ==
cust_id: string, city_count: bigint
Aggregate [cust_id#590], [cust_id#590, count(distinct city#600) AS city_count#826L]
+- Relation [cust_id#590,start_date#591,end_date#592,txn_id#593,date#594,year#595,month#596,day#597,expense_type#598,amt#599,city#600] parquet

== Optimized Logical Plan ==
Aggregate [cust_id#590], [cust_id#590, count(distinct city#600) AS city_count#826L]
+- Project [cust_id#590, city#600]
   +- Relation [cust_id#590,start_date#591,end_date#592,txn_id#593,date#5